<a href="https://colab.research.google.com/github/yuto-kobayashi-1/signate-AIQuest/blob/develop/210716_AIQuest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import lightgbm as lgb
from copy import deepcopy
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
data = pd.read_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/train.csv",encoding="utf_8")
test = pd.read_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/test.csv",encoding="utf_8")

In [148]:
data.head()

,id,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode,y
0,0,6,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",2.0,Real Bed,1.0,4.0,flexible,LA,t,My place is meant for family and a few friends...,2016-07-27,t,f,NaN,2016-07-13,f,2016-07-27,33.788931,-118.154761,The Penthouse,NaN,1,Apartment,60.0,Private room,NaN,90804,138.0
1,1,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,DC,t,This is a new listing for a lovely guest bedro...,2016-09-12,t,t,100%,2015-12-30,f,2017-03-31,38.934810,-76.978190,Guest Bedroom in Brookland,Brookland,9,House,100.0,Private room,https://a0.muscache.com/im/pictures/e4d8b51f-6...,20018,42.0
2,2,2,"{TV,Internet,""Wireless Internet"",Kitchen,""Indo...",2.0,Real Bed,1.0,1.0,strict,NYC,t,We're looking forward to your stay at our apt....,2016-06-15,t,f,100%,2016-05-21,t,2017-08-13,40.695118,-73.926240,Clean Modern Room in Lux Apt 1 Block From J Train,Bushwick,27,Apartment,83.0,Private room,https://a0.muscache.com/im/pictures/5ffecc9b-d...,NaN,65.0
3,3,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,SF,t,BEST CITY VIEWS - - ROOF DECK W/ BBQ & WiFi - ...,2014-03-15,t,t,100%,2012-06-19,t,2017-09-03,37.796728,-122.411906,BEST views + reviews! 5/5 stars*****,Nob Hill,38,Apartment,95.0,Private room,NaN,94133,166.0
4,4,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,Real Bed,1.0,1.0,strict,NYC,t,Charming Apartment on the upper west side of M...,2015-08-05,t,t,100%,2015-03-25,f,2017-09-10,40.785050,-73.974691,Charming 1-bedroom - UWS Manhattan,Upper West Side,5,Apartment,100.0,Entire home/apt,https://a0.muscache.com/im/pictures/92879730/5...,10024,165.0


In [149]:
def preprocessing(df):
  # df["host_response_rate"] = df["host_response_rate"].fillna('0%')
  df["host_response_rate"] = df["host_response_rate"].str.replace('%', '').astype(float)

  return df

def label_encorder(df_train,df_test,cols):
  df_train["flg"] = 0
  df_test["y"] = 100
  df_test["flg"] = 1

  df_ = pd.concat([df_train,df_test]).reset_index(drop=True)
  le = LabelEncoder()

  for col in cols:
    df_[col] = df_[col].fillna("OK")
    null_list = list(df_[df_[col] == "OK"].index)
    print(len(null_list))
    encoded = le.fit_transform(df_[col].values)
    decoded = le.inverse_transform(encoded)
    df_[col] = encoded
    df_.loc[null_list,col]=np.nan

  df_train_ = df_[df_["flg"]==0].drop("flg",axis=1)
  df_test_ = df_[df_["flg"]==1].drop(["flg","y"],axis=1)

  return df_train_ ,  df_test_

def train(X,y,label_cols):

  kf = KFold(n_splits=5,shuffle=True,random_state=0)

  models = []
  for train_index, test_index in kf.split(X, y):
    X_train = X.iloc[train_index]
    y_train =  y.iloc[train_index]
    X_valid = X.iloc[test_index]
    y_valid = y.iloc[test_index]

    X_train, y_train, X_valid, y_valid =  target_encording(X_train, y_train, X_valid, y_valid,label_cols)

    params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 64,
    'min_data_in_leaf': 20,
    'max_depth': 7,
    'verbose': 0,
  }
    train_data = lgb.Dataset(
      data=X_train, 
      label=y_train,
  )

    validation_data = lgb.Dataset(
      data=X_valid, 
      label=y_valid, 
  )

    model = lgb.train(
      params=params, 
      train_set=train_data, 
      num_boost_round=1000, 
      early_stopping_rounds=100,
      valid_sets=[validation_data]
  )

    models.append(model)

  return models 

def feature_importance(models,data,cols):
  importance = pd.DataFrame(columns=data[cols].columns)

  for model in models:
    imp_ = pd.Series(model.feature_importance(),index=data[cols].columns)
    importance = importance.append(imp_,ignore_index=True)

  importance = importance.mean()

  importance = pd.DataFrame(importance,columns={"importance"}).sort_values("importance",ascending=False)

  importance = importance/importance.sum()
  return importance

In [150]:
def target_encording(tr_x, tr_y, va_x, va_y, cols):

  for c in cols:

    data_tmp = pd.DataFrame({c:tr_x[c],'target':tr_y})
    target_mean = data_tmp.groupby(c)['target'].mean()
    #バリデーションデータのカテゴリを置換
    va_x.loc[:,c] = va_x[c].map(target_mean)

    #学習データの変換後の値を格納する配列を準備
    tmp = np.repeat(np.nan,tr_x.shape[0])
    kf_encording = KFold(n_splits=5,shuffle=True,random_state=72)
    for idx_1, idx_2 in kf_encording.split(tr_x):
      #out-of-foldで各カテゴリにおける目的変数の平均を計算
      target_mean = data_tmp.iloc[idx_1].groupby(c)['target'].mean()
      #変換後の値を一時配列に格納
      tmp[idx_2] = tr_x[c].iloc[idx_2].map(target_mean)

    tr_x.loc[:,c] = tmp

  return tr_x, tr_y, va_x, va_y

def target_encording_test(tr_x, tr_y, va_x, cols):
  for c in cols:
    data_tmp = pd.DataFrame({c:tr_x[c],'target':tr_y})
    target_mean = data_tmp.groupby(c)['target'].mean()
    #バリデーションデータのカテゴリを置換
    va_x.loc[:,c] = va_x[c].map(target_mean)

  return va_x

def predict(models,df):
  preds = np.zeros((len(test),len(models)))
  for n , model in enumerate(models):
    # pred = model.predict(df_test[cols],num_iteration=model.best_iteration)
    pred = model.predict(df_test[cols],num_iteration=model.best_iteration)
    preds[:,n] = pred

  score = pd.DataFrame()
  score["id"] = df_test["id"].astype(int)
  score["pred"] = np.mean(preds,axis=1)
  
  return score

In [151]:
len(data)

55583

In [152]:
len(test)

18528

In [153]:
cols = ['accommodates','bathrooms', 'bed_type', 'bedrooms',
       'beds', 'cancellation_policy', 'city', 'cleaning_fee',
       'host_has_profile_pic', 'host_identity_verified',
       'instant_bookable',
       'latitude', 'longitude', 'neighbourhood', 'number_of_reviews',
       'property_type', 'review_scores_rating', 'room_type',"host_response_rate"]


label_cols = ['bed_type','cancellation_policy',"city","cleaning_fee","host_has_profile_pic","host_identity_verified","instant_bookable","neighbourhood","property_type","room_type"]


In [154]:
data = preprocessing(data)
test = preprocessing(test)

In [155]:
# df_train , df_test  = label_encorder(data,test,label_cols)
df_train , df_test  = data,test

In [156]:
models = train(df_train[cols],df_train['y'],label_cols)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


[1]	valid_0's rmse: 159.326
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 152.193
[3]	valid_0's rmse: 145.917
[4]	valid_0's rmse: 140.699
[5]	valid_0's rmse: 136.27
[6]	valid_0's rmse: 132.524
[7]	valid_0's rmse: 129.469
[8]	valid_0's rmse: 126.905
[9]	valid_0's rmse: 124.534
[10]	valid_0's rmse: 122.59
[11]	valid_0's rmse: 120.964
[12]	valid_0's rmse: 119.521
[13]	valid_0's rmse: 118.452
[14]	valid_0's rmse: 117.384
[15]	valid_0's rmse: 116.618
[16]	valid_0's rmse: 115.998
[17]	valid_0's rmse: 115.45
[18]	valid_0's rmse: 114.85
[19]	valid_0's rmse: 114.4
[20]	valid_0's rmse: 114.03
[21]	valid_0's rmse: 113.689
[22]	valid_0's rmse: 113.322
[23]	valid_0's rmse: 113.076
[24]	valid_0's rmse: 112.89
[25]	valid_0's rmse: 112.534
[26]	valid_0's rmse: 112.338
[27]	valid_0's rmse: 112.197
[28]	valid_0's rmse: 112.025
[29]	valid_0's rmse: 111.889
[30]	valid_0's rmse: 111.819
[31]	valid_0's rmse: 111.699
[32]	valid_0's rmse: 111.6
[33]	valid_0's rmse: 111.506

In [157]:
aucs=[]
for model in models:
  aucs.append(model.best_score["valid_0"]["rmse"])
np.mean(aucs)

106.95243918555903

In [158]:
importance = feature_importance(models,data,cols)
importance

,importance
neighbourhood,0.145596
longitude,0.118908
latitude,0.095022
accommodates,0.071880
bathrooms,0.065952
number_of_reviews,0.059379
property_type,0.055981
review_scores_rating,0.055659
host_response_rate,0.054592
bedrooms,0.044696


In [159]:
df_test = target_encording_test(df_train[cols],df_train['y'],df_test,label_cols)

In [160]:
df_test[cols].head()

,accommodates,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,host_has_profile_pic,host_identity_verified,instant_bookable,latitude,longitude,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,host_response_rate
0,6,2.0,161.855415,2.0,2.0,177.708076,165.629287,163.180471,160.028517,162.540752,152.712832,42.359278,-71.069962,201.766234,58,187.191627,90.0,218.635896,100.0
1,3,1.0,161.855415,1.0,1.0,141.202828,154.555324,163.180471,160.028517,159.091831,162.805405,34.084747,-118.367355,162.843985,4,125.772959,100.0,218.635896,100.0
2,2,1.0,161.855415,0.0,1.0,149.785029,143.096152,151.821298,160.028517,159.091831,162.805405,40.720541,-73.959192,138.342622,0,146.586080,NaN,88.334405,100.0
3,4,1.0,161.855415,1.0,2.0,177.708076,143.096152,151.821298,160.028517,159.091831,162.805405,40.681117,-73.944091,101.388235,0,146.586080,NaN,218.635896,NaN
4,3,1.5,161.855415,1.0,2.0,177.708076,154.555324,163.180471,160.028517,159.091831,162.805405,34.150995,-118.409359,NaN,6,202.257112,92.0,218.635896,100.0


In [161]:
score = predict(models,df_test[cols])

In [162]:
score

,id,pred
0,0,270.185237
1,1,130.681085
2,2,104.775816
3,3,149.930482
4,4,147.001164
...,...,...
18523,18523,192.778096
18524,18524,135.002120
18525,18525,197.904333
18526,18526,130.706709


In [163]:
score.dtypes

id        int64
pred    float64
dtype: object

In [164]:
score.to_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/submit.csv",header=False,index=False)